<a href="https://colab.research.google.com/github/Sayara20/LimpiezaDatos/blob/main/SismosMXN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from transformers import pipeline
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import unicodedata
import re
import nltk


In [ ]:
#CArgar archivo
df = pd.read_csv("sismos.csv", encoding='utf-8', sep=',')
df.head()

,Fecha,Hora,Magnitud,Latitud,Longitud,Profundidad,Referencia de localizacion,Fecha UTC,Hora UTC,Estatus
0,01/01/2026,00:06:31,1.4,23.042,-109.741,10.5,"4 km al SUROESTE de SAN JOSE DEL CABO, BCS",01/01/2026,06:06:31,verificado
1,01/01/2026,00:06:56,2.8,16.814,-98.307,12.2,"18 km al NORESTE de OMETEPEC, GRO",01/01/2026,06:06:56,verificado
2,01/01/2026,00:09:27,3.6,17.398,-98.444,58.5,"22 km al SURESTE de TLAPA, GRO",01/01/2026,06:09:27,verificado
3,01/01/2026,00:40:15,3.0,16.810,-98.312,10.3,"17 km al NORESTE de OMETEPEC, GRO",01/01/2026,06:40:15,verificado
4,01/01/2026,01:11:25,3.5,16.524,-95.804,62.9,"64 km al NOROESTE de TEHUANTEPEC, OAX",01/01/2026,07:11:25,verificado


In [ ]:
#total de registros
print(len(df))


5087


#EDA

In [ ]:
#Hacemos copia de seguridad de nuestra BD
df_copy = df.copy()
df_copy.head()

,Fecha,Hora,Magnitud,Latitud,Longitud,Profundidad,Referencia de localizacion,Fecha UTC,Hora UTC,Estatus
0,01/01/2026,00:06:31,1.4,23.042,-109.741,10.5,"4 km al SUROESTE de SAN JOSE DEL CABO, BCS",01/01/2026,06:06:31,verificado
1,01/01/2026,00:06:56,2.8,16.814,-98.307,12.2,"18 km al NORESTE de OMETEPEC, GRO",01/01/2026,06:06:56,verificado
2,01/01/2026,00:09:27,3.6,17.398,-98.444,58.5,"22 km al SURESTE de TLAPA, GRO",01/01/2026,06:09:27,verificado
3,01/01/2026,00:40:15,3.0,16.810,-98.312,10.3,"17 km al NORESTE de OMETEPEC, GRO",01/01/2026,06:40:15,verificado
4,01/01/2026,01:11:25,3.5,16.524,-95.804,62.9,"64 km al NOROESTE de TEHUANTEPEC, OAX",01/01/2026,07:11:25,verificado


In [ ]:
#Obtenemos el estado

df_copy['Estado'] =df_copy['Referencia de localizacion'].str.split(',').str[-1].copy().str.strip()
df_copy['timestamp'] = pd.to_datetime(df_copy['Fecha'], dayfirst=True).astype('int64')
df_copy.head()

,Fecha,Hora,Magnitud,Latitud,Longitud,Profundidad,Referencia de localizacion,Fecha UTC,Hora UTC,Estatus,Estado,Estado_ID,timestamp
0,01/01/2026,00:06:31,1.4,23.042,-109.741,10.5,"4 km al SUROESTE de SAN JOSE DEL CABO, BCS",01/01/2026,06:06:31,verificado,BCS,1,1767225600000000000
1,01/01/2026,00:06:56,2.8,16.814,-98.307,12.2,"18 km al NORESTE de OMETEPEC, GRO",01/01/2026,06:06:56,verificado,GRO,6,1767225600000000000
2,01/01/2026,00:09:27,3.6,17.398,-98.444,58.5,"22 km al SURESTE de TLAPA, GRO",01/01/2026,06:09:27,verificado,GRO,6,1767225600000000000
3,01/01/2026,00:40:15,3.0,16.810,-98.312,10.3,"17 km al NORESTE de OMETEPEC, GRO",01/01/2026,06:40:15,verificado,GRO,6,1767225600000000000
4,01/01/2026,01:11:25,3.5,16.524,-95.804,62.9,"64 km al NOROESTE de TEHUANTEPEC, OAX",01/01/2026,07:11:25,verificado,OAX,13,1767225600000000000


In [ ]:
#Valores unicos de sismos por región
cols_cat = ['Estado']
for col in cols_cat:
    print(f'{col}:{df_copy[col].unique()}')


Estado:['BCS' 'GRO' 'OAX' 'SIN' 'BC' 'CHIS' 'ZAC' 'PUE' 'SON' 'MICH' 'HGO' 'VER'
 'COL' 'JAL' 'MEX' 'SLP' 'TAB' 'CHIH' 'NL' 'MOR' 'TLAX' 'DGO' 'TAMS']


In [ ]:
#Agrupar sismos
df_copy.groupby('Estado', as_index=False)['Magnitud'].count()


,Estado,Magnitud
0,BC,268
1,BCS,345
2,CHIH,4
3,CHIS,534
4,COL,104
5,DGO,1
6,GRO,2815
7,HGO,5
8,JAL,107
9,MEX,1


In [ ]:
#Ordenar de mayor a menor y quitar el indice sobre la columna Estado para pasarla como una columna del DF
df_copy.groupby('Estado')['Magnitud'].count().reset_index().sort_values(by="Magnitud" , ascending=False)

,Estado,Magnitud
6,GRO,2815
13,OAX,597
3,CHIS,534
1,BCS,345
0,BC,268
10,MICH,116
8,JAL,107
4,COL,104
21,VER,86
22,ZAC,35


In [ ]:
!pip install plotly

In [ ]:
import plotly.express as px

# Preparamos el resumen: contamos profundidad por estado y ordenamos
df_plot = df_copy.groupby('Estado')['Profundidad'].count().reset_index()
df_plot = df_plot.sort_values(by='Profundidad', ascending=False)

In [ ]:
fig = px.bar(df_plot,
             x='Estado',
             y='Profundidad',
             title='Conteo de Profundidad por Estado',
             labels={'Profundidad': 'Número de Registros', 'Estado': 'Estado'},
             color='Profundidad', # Añade un degradado de color según el valor
             color_continuous_scale='Viridis')

fig.show()

In [ ]:
#graficar cómo varía la profundidad en cada estado (mínimos, máximos y promedios)
fig = px.box(df_copy, x='Estado', y='Profundidad', color='Estado', title='Distribución de Profundidad por Estado')
fig.show()

In [ ]:
df_copy.head()

,Fecha,Hora,Magnitud,Latitud,Longitud,Profundidad,Referencia de localizacion,Fecha UTC,Hora UTC,Estatus,Estado,Estado_ID,timestamp
0,01/01/2026,00:06:31,1.4,23.042,-109.741,10.5,"4 km al SUROESTE de SAN JOSE DEL CABO, BCS",01/01/2026,06:06:31,verificado,BCS,1,1767225600000000000
1,01/01/2026,00:06:56,2.8,16.814,-98.307,12.2,"18 km al NORESTE de OMETEPEC, GRO",01/01/2026,06:06:56,verificado,GRO,6,1767225600000000000
2,01/01/2026,00:09:27,3.6,17.398,-98.444,58.5,"22 km al SURESTE de TLAPA, GRO",01/01/2026,06:09:27,verificado,GRO,6,1767225600000000000
3,01/01/2026,00:40:15,3.0,16.810,-98.312,10.3,"17 km al NORESTE de OMETEPEC, GRO",01/01/2026,06:40:15,verificado,GRO,6,1767225600000000000
4,01/01/2026,01:11:25,3.5,16.524,-95.804,62.9,"64 km al NOROESTE de TEHUANTEPEC, OAX",01/01/2026,07:11:25,verificado,OAX,13,1767225600000000000


#Seleccion de variable

In [ ]:
# Convertimos el Estado (texto) a números para que el modelo lo entienda (Label Encoding)
df_copy['Estado_ID'] = df_copy['Estado'].astype('category').cat.codes

print(f"Estados detectados: {df_copy['Estado'].unique()}")

Estados detectados: ['BCS' 'GRO' 'OAX' 'SIN' 'BC' 'CHIS' 'ZAC' 'PUE' 'SON' 'MICH' 'HGO' 'VER'
 'COL' 'JAL' 'MEX' 'SLP' 'TAB' 'CHIH' 'NL' 'MOR' 'TLAX' 'DGO' 'TAMS']


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

#Entrenamiento

In [ ]:
# Definimos X con la nueva columna
X = df_copy[['Latitud', 'Longitud', 'Profundidad', 'Estado_ID','Magnitud']]
y = df_copy['timestamp']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el bosque
modelo = RandomForestRegressor(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

# Obtener importancia
importancias = pd.DataFrame({
    'Variable': X.columns,
    'Importancia': modelo.feature_importances_
}).sort_values(by='Importancia', ascending=False)

#Vamos a ver si el Estado resultó ser más importante que la Profundidad.

In [ ]:
fig = px.bar(importancias,
             x='Importancia',
             y='Variable',
             orientation='h',
             title='Importancia de Variables (Incluyendo Estado)',
             color='Variable',
             color_discrete_sequence=px.colors.qualitative.Pastel,
             template='plotly_white')

fig.show()

#Simulador de magnitud


In [ ]:
# Creamos un diccionario para mapear el nombre del estado a su ID
mapeo_estados = dict(zip( df_copy['Estado'],  df_copy['Estado_ID']))

def predecir_magnitud(lat, lon, prof, estado_nombre,magnitud):
    # Verificamos si el estado existe en nuestros datos
    if estado_nombre not in mapeo_estados:
        return f"Estado '{estado_nombre}' no reconocido. Prueba con: {list(mapeo_estados.keys())[:5]}"

    id_estado = mapeo_estados[estado_nombre]

    # El modelo espera un DataFrame o array 2D


    datos_entrada = pd.DataFrame([[lat, lon, prof, id_estado,magnitud]],
                                 columns=['Latitud', 'Longitud', 'Profundidad', 'Estado_ID','Magnitud'])

    prediccion = modelo.predict(datos_entrada)[0]
    return round(prediccion, 2)

#probando el simulador


In [ ]:
# Ejemplo 1: Sismo en Guerrero a 10km de profundidad
mag1 = predecir_magnitud(16.8, -99.5, 10.0, 'GRO',5.5)
print(f"Predicción para Guerrero (Superficial): {mag1} Mw")

# Ejemplo 2: Sismo en Chiapas a 120km de profundidad
mag2 = predecir_magnitud(17.2, -94.3, 120.0, 'CHIS',5.5)
print(f"Predicción para Chiapas (Profundo): {mag2} Mw")

Predicción para Guerrero (Superficial): 1.768019616e+18 Mw
Predicción para Chiapas (Profundo): 1.768596768e+18 Mw


In [ ]:
y_pred = modelo.predict(X_test)

fig = px.scatter(x=y_test, y=y_pred,
                 labels={'x': 'Magnitud Real', 'y': 'Magnitud Predicha'},
                 title='Precisión del Modelo: Real vs. Predicion',
                 opacity=0.5,
                 color_discrete_sequence=['#B5EAD7']) # Tono pastel menta

# Añadimos una línea de referencia
fig.add_shape(type="line", x0=y.min(), y0=y.min(), x1=y.max(), y1=y.max(),
              line=dict(color="LightSeaGreen", dash="dash"))

fig.show()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predicción
preds = model.predict(X_test)
print(f"Error cuadrático medio: {mean_squared_error(y_test, preds)}")

Error cuadrático medio: 5.8009697983322035e+29
